In [60]:
#import
import numpy as np
import nltk
import string
from nltk.tokenize import TweetTokenizer
import spell_check
from wordsegment import load, segment
from collections import Counter
from nltk.stem.snowball import SnowballStemmer

In [61]:
pos = [line.rstrip('\n') for line in open('../data/train_pos.txt')]
pos_org = pos.copy()

In [87]:
def upload_dict(file, dup=False, order=False):
    '''if dup, duplicate smiley set by adding -
    if order, take first half as keys'''
    path = '../data/'+file
    word_list = [line.rstrip('\n') for line in open(path)]
    keys = np.asarray(word_list)[2*np.arange(int(len(word_list)/2))]
    keys = [k.strip() for k in keys]
    values = np.asarray(word_list)[2*np.arange(int(len(word_list)/2))+1]
    word_dict = dict(zip(keys, values))
    if dup:
        keys2 = [k.replace('-','') for k in keys]
        word_dict = dict(zip(keys+keys2, values+values))
    return word_dict

def spell_check_tweets(tweet_list):
    '''Correct words'''
    load()
    new_pos = []
    for tweet in tweet_list:
        words = [spell_check.correction(word) for word in tokenize(tweet)]
        new_pos.append(' '.join(word for word in words))
    return new_pos

def replace_neg_verbs(tweet_list):
    #Verbs dataset
    verb_dict = upload_dict('neg_verbs.txt')
    
    new_pos = []
    for tweet in tweet_list:
        words = tokenize(tweet)
        for i in range(len(words)):
            if words[i] in verb_dict.keys():
                words[i] = verb_dict[words[i]]
        new_pos.append(' '.join(word for word in words))
    return new_pos
    

def remove_punctuation(tweet_list):
    '''Remove all punctuations from string'''
    new_pos = []
    for tweet in tweet_list:
        new_pos.append("".join(l for l in tweet if l not in string.punctuation))
    return new_pos

def tokenize(text):
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def remove_stopwords(tweet_list):
    '''Remove all stopwords'''
    stop_words = [line.rstrip('\n').lower() for line in open('../data/stopwords.txt')]
    
    new_pos = []
    for tweet in tweet_list:
        words = [word for word in tokenize(tweet) if not word in stop_words]
        new_pos.append(' '.join(word for word in words))
    return new_pos

# delete whitespace
def replace_smiley_slang(tweet_list):
    #Smiley dataset
    smiley_dict = upload_dict('smiley.txt')
    
    #Slangs datasets 1
    slang_dict = upload_dict('slang.txt')
    
    #Slang dataset 2
    slang_dict2 = upload_dict('slang2.txt')
    
    new_pos = []
    for tweet in tweet_list:
        words = tokenize(tweet)
        for i in range(len(words)):
            if words[i] in smiley_dict.keys():
                words[i] = smiley_dict[words[i]]
            elif words[i] in slang_dict.keys():
                words[i] = slang_dict[words[i]]
            elif words[i] in slang_dict2.keys():
                words[i] = slang_dict2[words[i]]
        new_pos.append(' '.join(word for word in words))
    return new_pos

In [108]:
try:
    pos = [line.rstrip('\n') for line in open('../data/train_pols.txt')]
except:SyntaxError
    print('file not found')

IndentationError: unexpected indent (<ipython-input-108-fbe28d3dffe3>, line 4)

In [96]:
# Replace neg_verbs and slogans
pos = replace_neg_verbs(pos[:100])
pos

['dunno justin read mention not justin god knows hope follow believe 15',
 'logic significant dumb will not even crop name photo tsk',
 'put casper box looved battle crakkbitch',
 'thanks sir cant trip lil mama keep doin ya thang',
 'visiting brother tmr bestest birthday gift eveerrr',
 'yay lifecompleted tweet facebook let know please',
 '1dnextalbumtitle feel rollercoaster life song cocept life yolo becoming famous Lovefriendship 3s bigger heart 14 followmeplz Lovefriendship 3s bigger heart x15',
 'workin hard hardly workin hardees future coworker',
 'saw ill replying bit',
 'belong']

In [92]:
# Replace smileys and slogans
pos = replace_smiley_slang(pos)
len(pos)

10

In [97]:
# Remove punctiuations
pos = remove_punctuation(pos)
pos[1]

'logic significant dumb will not even crop name photo tsk'

In [99]:
# Remove stopWords
pos = remove_stopwords(pos)
pos[1]

'logic significant dumb not even crop name photo tsk'

In [100]:
## Spell check
#import time
#t =time.time()
#pos_f = []
#pos_f = spell_check_tweets(pos)
#t = time.time()-t
#pos_f = remove_stopwords(pos_f)

In [102]:
for idx in range(10):
    print(':::::::::::', idx)
    print(pos_org[idx])
    print(pos[idx])

::::::::::: 0
<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
dunno justin read mention not justin god knows hope follow believe 15
::::::::::: 1
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
logic significant dumb not even crop name photo tsk
::::::::::: 2
" <user> just put casper in a box ! " looved the battle ! #crakkbitch
put casper box looved battle crakkbitch
::::::::::: 3
<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
thanks sir cant trip lil mama keep doin ya thang
::::::::::: 4
visiting my brother tmr is the bestest birthday gift eveerrr ! ! !
visiting brother tmr bestest birthday gift eveerrr
::::::::::: 5
<user> yay ! ! #lifecompleted . tweet / facebook me to let me know please
yay lifecompleted tweet facebook let know please
::::::::::: 6
<user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : l

In [72]:
def separate(words):
    return ' '.join(word for word in segment(words))

In [76]:
def segmentation(pos):
    sep = []
    dictionary = Counter(tokenize(open('../data/english_words.txt').read()))
    for tweet in pos:
        tokens = tokenize(tweet)
        for i in range(len(tokens)):
            if tokens[i] not in dictionary:
                tokens[i] = separate(tokens[i])
        sep.append(' '.join(word for word in tokens))
    return pos

In [77]:
sep

['dunno justin read mention not justin god knows hope follow believe 15',
 'logic significant dumb wont even crop name photo tsk',
 'put casper box loove d battle crakk bitch',
 'thanks sir dont trip lil mama keep do in ya thang',
 'visiting brother tmr bestest birthday gift eve errr',
 'yay life completed tweet facebook let know please',
 '1d next album title feel rollercoaster life song co cept life yolo becoming famous love friendship 3s bigger heart 14 follow me plz love friendship 3s bigger heart x15',
 'work in hard hardly work in hardees future coworker',
 'saw ill replying bit',
 'belong',
 'and dd cheer nationals 2013',
 'send invitation shop online find everything need without leaving home',
 'woke finna go church',
 'agreed 12 days left tho',
 'monet kate melo',
 'like dam mm lexis got lot say ur laugh loud',
 'grateful today dream fulfilled heart significant full first 3 completed tracks arrived back new york yes lord',
 'home affairs shall later',
 'barca bout beat real ma

In [ ]:
pos_f = spell_check_tweets(pos[10:20])

In [ ]:
pos_f[0]

In [ ]:
a = [1,2,3,4]
b = []
(b or a)

In [ ]:
stemmer = SnowballStemmer("english")

In [ ]:
stemmer.stem("wasnt")

In [ ]:
spell_check.correction('wasnt')

In [ ]:
load()
spell_check.correction('konneczion')

In [ ]:
pos[0]

In [32]:
# Remove punctiuation
pos_tokens = [tokenize(tweet) for tweet in pos]

In [33]:
flat_pos = [word for tweet in pos_tokens for word in tweet]

In [ ]:
'fml' in flat_pos

In [ ]:
counter['yw']

In [34]:
counter = Counter(flat_pos)

In [35]:
counter.most_common()

[('user', 81660),
 ('i', 37442),
 ('a', 26296),
 ('significant', 9466),
 ('im', 9346),
 ('url', 9331),
 ('love', 8587),
 ('rt', 7422),
 ('good', 5724),
 ('laugh', 5717),
 ('follow', 5683),
 ('loud', 5464),
 ('u', 4531),
 ('day', 4160),
 ('x', 4128),
 ('heart', 3643),
 ('haha', 3418),
 ('bigger', 3284),
 ('friendship', 3259),
 ('back', 3256),
 ('Love', 3229),
 ('3s', 3229),
 ('dont', 3125),
 ('today', 3063),
 ('ill', 2958),
 ('time', 2653),
 ('beautiful', 2576),
 ('happy', 2520),
 ('smile', 2211),
 ('texting', 2163),
 ('make', 2115),
 ('youre', 2036),
 ('xx', 1976),
 ('fans', 1969),
 ('overheard', 1864),
 ('hope', 1835),
 ('tomorrow', 1816),
 ('great', 1793),
 ('yeah', 1704),
 ('hey', 1701),
 ('wait', 1624),
 ('nice', 1588),
 ('night', 1561),
 ('people', 1546),
 ('twitter', 1525),
 ('To', 1524),
 ('birthday', 1522),
 ('girl', 1502),
 ('tweet', 1482),
 ('gonna', 1392),
 ('ass', 1367),
 ('guys', 1324),
 ('morning', 1306),
 ('xxx', 1282),
 ('tonight', 1233),
 ('work', 1216),
 ('o', 1132),


In [ ]:
stop_words = ['user',  i', 'you', ]

In [138]:
def write_to_file(tweet_list, file):
	path = '../data/filterd_data/filterd_' + file
	f = open(str(path), 'w')
	for tweet in tweet_list:
		f.write(tweet+'\n')
	f.close()

In [139]:
write_to_file(pos, 'test.txt')

In [123]:
f= open("guru99.txt","w+")

In [ ]:
a = []
for word in flat_pos:
    if word[-2:] =="n t": a.append(word)

In [ ]:
Counter(a).most_common()

In [ ]:
for p in pos[10:20]:
    print('##', ' '.join(s for s in p))

In [ ]:
'201a5'.isdigit()

In [ ]:
spell_check.correction('abcd105')

In [ ]:
words_list = [line.rstrip('\n').lower() for line in open('../data/english_words.txt')]

In [ ]:
def dict_check(s):
    return s in words_list

In [ ]:
check = []
for tweet in pos[:10]:
    print(tweet)
    check.append([w for w in tweet if not dict_check(w)])

In [26]:
from nltk.corpus import stopwords

In [27]:
stopWords = set(stopwords.words('english'))

{'has', 'themselves', 'from', 'down', 'wouldn', 'me', 'herself', 'after', 'above', 'how', 'was', 'wasn', 'should', 'further', 'on', 'just', 'against', 'some', 'm', 'yours', 'once', 'don', 'only', 'under', 'off', 'you', 'before', 'her', 'at', 'why', 'were', 'having', 'same', 'hasn', 'here', 'ours', 'itself', 'o', 'both', 'i', 'ourselves', 'myself', 'them', 'very', 'he', 'then', 'who', 'below', 'mightn', 'those', 'through', 'nor', 'what', 'into', 'hadn', 'while', 'needn', 'their', 'had', 'are', 'in', 'during', 'most', 'mustn', 'over', 'll', 'where', 'is', 'between', 'again', 'our', 'it', 'as', 'an', 'being', 'not', 't', 'its', 'about', 'any', 'did', 'haven', 'until', 'am', 'shan', 'his', 'than', 'all', 'there', 'yourselves', 'or', 'so', 's', 'when', 'to', 'other', 'do', 'my', 'each', 'hers', 'whom', 'y', 'by', 'no', 'ain', 'been', 'because', 'him', 'which', 'if', 'aren', 'yourself', 'have', 'and', 'won', 'weren', 'out', 'a', 'up', 'doing', 've', 'with', 'shouldn', 'they', 'of', 'for', 'c